In [5]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append(os.path.abspath("../../"))
sys.path.append(os.path.abspath("../core"))
sys.path.append("/mnt/ssd/hyun/experiment/GAN_n2n/core")
%load_ext autoreload
%autoreload 2

In [6]:
import torch
import torch.nn as nn
from edge_core.model import Generator
from core.median_filter import *
from core.median_filter import apply_median_filter_cpu
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [7]:
import h5py
img_dict = {}
with h5py.File("../../data/test_Samsung_SNU_patches_SET050607080910_divided_by_fnum_setnum.hdf5") as f:
    for key in f.keys():
        #print(key)
        img_dict[key] = {} # f[key]
        for subkey in f[key].keys():
            img_dict[key][subkey] = np.array(f[key][subkey])

In [8]:

from core.utils import TrdataLoader, TedataLoader
from arguments import get_args
from torch.utils.data import DataLoader
args = get_args(env='colab')
args.pge_weight_dir = None #pge_weight_dir
args.loss_function = "MSE_Affine"
args.noise_type = 'Poisson-Gaussian'
args.model_type = 'FBI_Net'
# args.set_num = '1'
args.data_name = 'Samsung'
args.data_type = 'Grayscale'
args.lr = 1e-3
args.num_layers = 17
args.num_filters = 64
args.crop_size = 256
args.debug = False
args.test_set = None
args.integrate_all_set = True
args.individual_noisy_input = True
args.wholedataset_version = 'v2'
args.x_f_num = 'F01'
args.y_f_num = 'F02'
args.batch_size = 1
args.seed = 0

import random
# control the randomness
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True


In [9]:

from core.loss_functions import mse_bias, mse_affine, emse_affine, mse_affine_with_tv
def determine_loss_and_output_type(args):
    if args.loss_function== 'MSE': # upper bound 1-1(optional)
        args.loss_fn = torch.nn.MSELoss()
        args.num_output_channel = 1
        args.output_type = "linear"
    elif args.loss_function == 'N2V': #lower bound
        args.loss_fn = mse_bias
        args.num_output_channel = 1
        args.output_type = "linear"
    elif args.loss_function == 'MSE_Affine': # 1st(our case upper bound)
        args.loss_fn = mse_affine
        args.num_output_channel = 2
        args.output_type = "linear"
    elif args.loss_function == 'EMSE_Affine':
        
        args.loss_fn = emse_affine
        args.num_output_channel = 2
    elif args.loss_function == 'MSE_Affine_with_tv':
        args.loss_fn = mse_affine_with_tv
        vnum_output_channel = 2
        args.output_type = "linear"
    return args
def get_X_hat(Z, output):

    X_hat = output[:,:1] * Z + output[:,1:]
        
    #print(X_hat[0])
    return X_hat

In [10]:
tr_file_path = "/mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/train_Samsung_SNU_patches_SET050607080910_divided_by_fnum_setnum.hdf5"
tr_data_loader = TrdataLoader(_tr_data_dir=tr_file_path,_args=args)
tr_data_loader = DataLoader(tr_data_loader, batch_size=args.batch_size, shuffle=True, num_workers=0, drop_last=True)

te_file_path = file_path = "/mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/test_Samsung_SNU_patches_SET050607080910_divided_by_fnum_setnum.hdf5"
te_data_loader = TedataLoader(_te_data_dir=te_file_path,args=args)
te_data_loader = DataLoader(te_data_loader, batch_size=1, shuffle=False, num_workers=0, drop_last=False)

test set None
3600 ['SET05', 'SET06', 'SET07', 'SET08', 'SET09', 'SET10']


===Tr loader SET05 F01 vs F02===
3600/3600  images are loaded
===Tr loader SET06 F01 vs F02===
7200/7200  images are loaded
===Tr loader SET07 F01 vs F02===
10800/10800  images are loaded
===Tr loader SET08 F01 vs F02===
14400/14400  images are loaded
===Tr loader SET09 F01 vs F02===
18000/18000  images are loaded
===Tr loader SET10 F01 vs F02===
21600/21600  images are loaded
noisy_arr :  (21600, 256, 256) pixel value range from 0.0 ~ 1.0
clean_arr :  (21600, 256, 256) pixel value range from 0.0 ~ 1.0
num of training patches :  21600
===Te loader SET05 ['F01', 'F02', 'F04', 'F08', 'F16', 'F32']  train on F01 vs F02===
===Te loader SET06 ['F01', 'F02', 'F04', 'F08', 'F16', 'F32']  train on F01 vs F02===
===Te loader SET07 ['F01', 'F02', 'F04', 'F08', 'F16', 'F32']  train on F01 vs F02===
===Te loader SET08 ['F01', 'F02', 'F04', 'F08', 'F16', 'F32']  train on F01 vs F02===
===Te loader SET09 ['F01', 'F02', 'F04', 'F08', 'F16', 'F32']  train on F01 vs F02===
===Te loader SET10 ['F01', 'F

In [13]:

args.loss_function = "MSE_Affine"
args.nepochs = 20
args = determine_loss_and_output_type(args)
model = Generator(dim=1, num_output_channel= args.num_output_channel, ngf_factor=3, depth=4).cuda()
# from core.model import New_model
# model = New_model(channel = 1, output_channel =  args.um_output_channel, 
#                                 filters = args.num_filters, num_of_layers=args.num_layers, case = args.model_type, 
#                                 BSN_type = {"type" : args.BSN_type, "param" : args.BSN_param},
#                                 output_type = args.output_type, sigmoid_value = args.sigmoid_value)
            


In [16]:
# from torchsummary import summary
# summary(model, (1, 256, 256))

ModuleNotFoundError: No module named 'torchsummary'

In [17]:
from tqdm.notebook import tqdm
from core.utils import get_PSNR,get_SSIM
def eval(args):
    psnr_arr = []
    ssim_arr = []
    loss_arr = []
    
    denoised_img_arr = []
    with torch.no_grad():
         for batch_idx, (source, target) in enumerate(te_data_loader):
            source = source.cuda()
            target = target.cuda()
            
            output = model(source)
            loss = args.loss_fn(output, target).cpu().numpy()
                        
            # Update loss
            if args.loss_function == 'MSE':
                output = output.cpu().numpy()
                X_hat = np.clip(output, 0, 1)
                X = target.cpu().numpy()
                
            elif args.loss_function[:10] == 'MSE_Affine':
                
                Z = target[:,:1]
                X = target[:,1:].cpu().numpy()
                X_hat = np.clip(get_X_hat(Z,output).cpu().numpy(), 0, 1)
                
            elif  args.loss_function == 'N2V':
                X = target[:,1:].cpu().numpy()
                X_hat = np.clip(output.cpu().numpy(), 0, 1)

            
            loss_arr.append(loss)
            psnr_arr.append(get_PSNR(X[0], X_hat[0]))
            ssim_arr.append(get_SSIM(X[0], X_hat[0],args.data_type))
            denoised_img_arr.append(X_hat[0].reshape(X_hat.shape[2],X_hat.shape[3]))
            print(f"Loss : {loss:.4f}, PSNR : {psnr_arr[-1]:.2f} dB, SSIM : {ssim_arr[-1]:.4f}")
            if batch_idx == 0:
                plt.subplot(131)
                plt.imshow(source[0], cmap='gray')
                plt.subplot(132)
                plt.title(f"PSNR : {psnr_arr[-1]:.2f} dB\nSSIM : {ssim_arr[-1]:.4f}")
                plt.imshow(target[0], cmap='gray')
                plt.subplot(133)
                plt.imshow(target[0,1:].cpu().numpy().reshape(X_hat.shape[2],X_hat.shape[3]), cmap='gray')
                plt.pause(0.01)
            break
    mean_loss = np.mean(loss_arr)
    mean_psnr = np.mean(psnr_arr)
    mean_ssim = np.mean(ssim_arr)

In [18]:

is_test = True
def train(args):
    
    optim = torch.optim.Adam(model.parameters(), lr=args.lr)
    mean_tr_loss = []
    for epoch in range(1,args.nepochs+1):
        print("====== Epoch %d ======" % epoch)
        tr_loss = []
        pbar = tqdm(enumerate(tr_data_loader),total=len(tr_data_loader))
        for batch_idx, (source, target) in pbar:

            optim.zero_grad()
            # if is_test is True:
            #     print(source.shape, target.shape)
                
            source = source.cuda()
            target = target.cuda()
            
            output = model(source)
            # print(output.shape)
            loss = args.loss_fn(output,target)
            # loss = loss + 
            loss.backward()
            optim.step()
            tr_loss.append(loss.detach().cpu().numpy())
            pbar.set_postfix(loss=loss.item())
        mean_tr_loss.append(np.mean(tr_loss))
        
        eval(args)
    return mean_tr_loss, (source, target, output)

In [33]:
import torch.nn as nn
mseloss = nn.MSELoss()
is_test = True
def train_constrained_weight_change(args):
    
    optim = torch.optim.Adam(model.parameters(), lr=args.lr)
    mean_tr_loss = []
    for epoch in range(1,args.nepochs+1):
        print("====== Epoch %d ======" % epoch)
        tr_loss = []
        pbar = tqdm(enumerate(tr_data_loader),total=len(tr_data_loader))
        for batch_idx, (src, tgt) in pbar:
                
            source = src.clone().cuda()
            target = tgt.cuda()
            
            output = model(source)
            model_weight = model.state_dict()
            # print(output.shape)
            loss_1st = args.loss_fn(output,target)
            loss_1st.backward(retain_graph=True)
            optim.step()
            optim.zero_grad()
            tr_loss.append(loss_1st)
            
            source = src.clone().cuda()
            output_updated = model(source)
            output_changed = mseloss(output_updated, output)
            loss_2nd = args.loss_fn(output,target) + args.lambda_weight * output_changed
            loss_2nd.backward()
            model.load_state_dict(model_weight)
            optim.step()
            optim.zero_grad()
            
            pbar.set_postfix(loss_1st=loss_1st.item(), loss_2nd = loss_2nd.item(),
                             output_changed = output_changed.item())
        mean_tr_loss.append(np.mean(tr_loss))
        
        eval(args)
    return mean_tr_loss, (source, target, output)

In [34]:
# epoch 5 ~ 7 정도에서 괜찮은 결과
# args.lr = 1e-3
# args.batch_size = 1
# epoch 3~5 정도에서 좋은 결과
# args.lr = 1e-3
# args.batch_size = 16
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
weights_init(model)
args.lr = 1e-4
args.batch_size = 16
# result = train(args)
args.lambda_weight = 1
result = train_constrained_weight_change(args)

====== Epoch 1 ======


  0%|          | 0/21600 [00:00<?, ?it/s]

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [2, 3, 1, 1]] is at version 8; expected version 7 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).